# DataCleaning by Daniel 
## In this ipynb, we will clean the data and create csv based on Su's document.
## Based on Su's document, we will clean the dataset focusing on the features "funder", "installer"and 'scheme_management'. 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'darkgrid')

### 1.0 Load data

In [17]:
#load traning data
train_values = pd.read_csv('data/training_set_values.csv')
train_labels = pd.read_csv('data/training_set_labels.csv')
# Merge these two datasets
df_train = train_values.merge(train_labels, on='id', how='inner')
# df_train.shape
#Load test data
df_test = pd.read_csv('data/test_set_values.csv')
df_test.shape

(14850, 40)

### 1.1 Check null value

In [3]:
# List number of null values
df_train.apply(lambda x: sum(x.isnull()))

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [4]:
df_train.shape

(59400, 41)

### 1.2 Drop 10+6+2 redundant columns

In [18]:
df_train = df_train.drop(columns = ['extraction_type','extraction_type_group','management','payment','quality_group','quantity_group','source_type','source','waterpoint_type'])
df_test = df_test.drop(columns = ['extraction_type','extraction_type_group','management','payment','quality_group','quantity_group','source_type','source','waterpoint_type'])
# There are too many null values in 'scheme name', so drop it
df_train = df_train.drop(columns = ['scheme_name'])
df_test = df_test.drop(columns = ['scheme_name'])
# df_train.shape

# drop wpt_name,num_private,subvillage,region_code,district_code,recorded_by

df_train = df_train.drop(columns = ['wpt_name'])
df_test = df_test.drop(columns = ['wpt_name'])
df_train = df_train.drop(columns = ['num_private'])
df_test = df_test.drop(columns = ['num_private'])
df_train = df_train.drop(columns=['subvillage'])
df_test = df_test.drop(columns=['subvillage'])
df_train = df_train.drop(columns = ['region_code', 'district_code'])
df_test = df_test.drop(columns = ['region_code', 'district_code'])
df_train = df_train.drop(columns = ['recorded_by'])
df_test = df_test.drop(columns = ['recorded_by'])

# Drop other geographical columns: 'lga', 'ward' 
df_train = df_train.drop(columns=['lga','ward'])
df_test = df_test.drop(columns=['lga','ward'])

### 1.3 fill nan
fill nan with 'unknown'

In [19]:
# Attention: there are 3334 null values in this column, not too much, replace NA by unknow
df_train['public_meeting'] = df_train['public_meeting'].fillna('unknown')
df_test['public_meeting'] = df_test['public_meeting'].fillna('unknown')

# Attention: 3056 NA, replace by unknown
df_train['permit'] = df_train['permit'].fillna('unknown')
df_test['permit'] = df_test['permit'].fillna('unknown')

### 1.4 Categories 'construction_year'

In [20]:
# df_train['permit'].isnull().sum()
def con_year_cleaning(df):
    year = df['construction_year'] 
    if year >= 1960 and year < 1970:
        return '60s'
    elif year >= 1970 and year < 1980:
        return '70s'
    elif year >= 1980 and year < 1990:
        return '80s'
    elif year >= 1990 and year < 2000:
        return '90s'
    elif year >= 2000 and year < 2010:
        return '00s'
    elif year >= 2010:
        return '10s'
    else:
        return 'unknown'
df_train['construction_year'] = df_train.apply(lambda row: con_year_cleaning(row), axis=1)
df_test['construction_year'] = df_test.apply(lambda row: con_year_cleaning(row), axis=1)
df_train['construction_year'].unique()

array(['90s', '10s', '00s', '80s', 'unknown', '70s', '60s'], dtype=object)

In [21]:
import re

In [87]:
# test for the manually categories the funder category
# some like "government/ community " are categories as government
# church has many 
# an idea: turn the 'nan' and '0' to be unknown 
# an idea: turn teh danida and danid as one value
# an idea: keep the values that count>500

def funder_cleaning(df):
    data = str(df['funder']).lower() # turn into string first
    pattern1 = 'gover.+'
    pattern2 = 'commu.+'
    pattern3 = 'priv.+'
    pattern4 = 'repub.+' # republic
    pattern5 = 'minis.+' # minist
    pattern6 = 'world.+' # big name 
    pattern7 = 'chur.+' # church
    pattern8 = 'coun.+' # council
    pattern9 = 'wate.+' # water: big name 
    pattern10 = 'dani.+' # church
    pattern7 = 'chur.+' # church
    if(re.search(pattern1, data)):
        return "government"
#         return data
    elif(re.search(pattern2, data)):
        return "community"
    elif(re.search(pattern3, data)):
        return 'individual'
    elif(re.search(pattern4, data)):
        return "government" # republic
    elif(re.search(pattern5, data)):
        return "government" # ministry 
    elif(re.search(pattern6, data)):
        return "community" # world bank, vision 
    elif(re.search(pattern7, data)):
        return "community" # church 
    elif(re.search(pattern8, data)):
        return "government" # council 
    elif(re.search(pattern9, data)):
        return "community" # water as a big name 
#     elif(re.search(pattern10, data)):
#         return data
    elif(data=='nan' or data=='0'):
        return "unknown" # danida as a top name 
    elif(data=='danida'):
        return 'danida'
    elif(data=='hesawa'):
        return 'hesawa'
    elif(data=='rwssp'):
        return 'rwssp'
    elif(data=='kkkt'):
        return 'kkkt'
    elif(data=='unicef'):
        return 'unicef'
    elif(data=='tasaf'):
        return 'tasaf'
    elif(data=='dhv'):
        return 'dhv'
    elif(data=='dwsp'):
        return 'dwsp'
    elif(data=='norad'):
        return 'norad'
    elif(data=='tcrs'):
        return 'tcrs'
    else:
        return "other"
#         return data
# df_train['funder'] = df_train.apply(lambda row: funder_cleaning(row), axis=1)
df_funder = df_train.apply(lambda row: funder_cleaning(row), axis=1)
df_funder.value_counts()

other         22861
government    11748
community      6317
unknown        4412
danida         3114
hesawa         2202
rwssp          1374
kkkt           1287
individual     1144
unicef         1057
tasaf           877
dhv             829
dwsp            811
norad           765
tcrs            602
dtype: int64

In [80]:
# based on the funder_cleaning 
# an idea: turn the 'nan' and '0' to be unknown 
# an idea: turn teh danida and danid as one value
# an idea: keep the values that count>500
def installer_cleaning(df):
    data = str(df['installer']).lower()
    pattern1 = 'gove.+'
    pattern2 = 'comm.+'
    pattern3 = 'priv.+'
    pattern4 = 'repub.+' # republic
    pattern5 = 'minis.+' # minist
    pattern6 = 'world.+' # big name 
    pattern7 = 'chur.+' # church
    pattern8 = 'coun.+' # council
    pattern9 = 'wate.+' # water: big name 
    pattern10 = 'dani.+' # church
    pattern7 = 'chur.+' # church
    if(re.search(pattern1, data)):
        return "government"
#         return data
    elif(re.search(pattern2, data)):
        return "community"
    elif(re.search(pattern3, data)):
        return 'individual'
    elif(re.search(pattern4, data)):
        return "government" # republic
    elif(re.search(pattern5, data)):
        return "government" # ministry 
    elif(re.search(pattern6, data)):
        return "community" # world bank, vision 
    elif(re.search(pattern7, data)):
        return "community" # church 
    elif(re.search(pattern8, data)):
        return "government" # council 
    elif(re.search(pattern9, data)):
        return "community" # water as a big name 
    elif(re.search(pattern10, data)):
        return "danida" # danida as a top name 
    elif(data=='nan' or data=='0'):
        return "unknown" # danida as a top name 
    elif(data=='dwe'):
        return 'dwe'
    elif(data=='hesawa'):
        return 'hesawa'
    elif(data=='rwe'):
        return 'rwe'
    elif(data=='kkkt'):
        return 'kkkt'
    elif(data=='tcrs'):
        return 'tcrs'
    elif(data=='ces'):
        return 'ces'
#         return data
    else:
        return "other"
#         return data
# df_train['funder'] = df_train.apply(lambda row: funder_cleaning(row), axis=1)
d_inst = df_train.apply(lambda row: installer_cleaning(row), axis=1)
d_inst.value_counts()

other         20857
dwe           17405
community      5436
government     4541
unknown        4432
danida         1678
hesawa         1395
rwe            1206
kkkt            910
tcrs            707
ces             610
individual      223
dtype: int64

### 1.5 Key Part: Data cleaning for features: funder,installer,scheme_management.
#### we still maintain top5 values using the lowercase of values and then set the other values as "other".

In [91]:
# d_fun = df_train.apply(lambda row: funder_cleaning(row), axis=1)
df_train['funder'] = df_train.apply(lambda row: funder_cleaning(row), axis=1)
df_test['funder'] = df_test.apply(lambda row: funder_cleaning(row), axis=1)
df_train['funder'].value_counts()

other         28417
government    11748
community      6317
danida         3114
hesawa         2202
rwssp          1374
kkkt           1287
unicef         1057
tasaf           877
dhv             829
dwsp            811
norad           765
tcrs            602
Name: funder, dtype: int64

In [90]:
# d_inst = df_train.apply(lambda row: installer_cleaning(row), axis=1)
# d_inst.value_counts()
df_train['installer'] = df_train.apply(lambda row: installer_cleaning(row), axis=1)
df_test['installer'] = df_test.apply(lambda row: installer_cleaning(row), axis=1)
df_train['installer'].unique()

array(['other', 'community', 'dwe', 'danida', 'government', 'kkkt', 'rwe',
       'hesawa', 'ces', 'tcrs'], dtype=object)

In [54]:
# 'scheme_management':One who operates the waterpoint.
df_train['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [12]:
# Keep top 5
def scheme_cleaning(df):
    data = str(df['scheme_management']).lower()
    if data == 'VWC'.lower():
        return 'vwc'
    elif data == 'WUG'.lower():
        return 'wug'
    elif data == 'Water authority'.lower():
        return 'water_authority'
    elif data == 'WUA'.lower():
        return 'wua'
    elif data == 'Water Board'.lower():
        return 'water_board'
    else:
        return 'other'

In [13]:
df_train['scheme_management'] = df_train.apply(lambda row: scheme_cleaning(row), axis=1)
df_train['scheme_management'].value_counts()

vwc                36793
other               8617
wug                 5206
water_authority     3153
wua                 2883
water_board         2748
Name: scheme_management, dtype: int64

# Note: (Uncomplete)
### 1.6 transform the time(mainly based on the date_recorded)
### This part from Zii

In [77]:
# just a try
# Transform the string date into days since recorded
# df_train['date_recorded'] = pd.to_datetime(df_train['date_recorded']) - pd.to_datetime(df_train['construction_year'])
# df_train.rename(columns = {'date_recorded':'days_since_recorded'}, inplace=True)
# df_test['date_recorded'] = pd.to_datetime(df_test['date_recorded']) - pd.to_datetime(df_test['construction_year'])
# df_test.rename(columns = {'date_recorded':'days_since_recorded'}, inplace=True)
# df_train['days_since_recorded'] = df_train['days_since_recorded'].astype('timedelta64[D]').astype(int)
# df_test['days_since_recorded'] = df_test['days_since_recorded'].astype('timedelta64[D]').astype(int)

In [16]:
df_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,basin,region,...,permit,construction_year,extraction_type_class,management_group,payment_type,water_quality,quantity,source_class,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,other,1390,Roman,34.938093,-9.856322,Lake Nyasa,Iringa,...,False,90s,gravity,user-group,annually,soft,enough,groundwater,communal standpipe,functional
1,8776,0.0,2013-03-06,other,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Mara,...,True,10s,gravity,user-group,never pay,soft,insufficient,surface,communal standpipe,functional
2,34310,25.0,2013-02-25,other,686,World vision,37.460664,-3.821329,Pangani,Manyara,...,True,00s,gravity,user-group,per bucket,soft,enough,surface,communal standpipe,functional
3,67743,0.0,2013-01-28,other,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,...,True,80s,submersible,user-group,never pay,soft,dry,groundwater,communal standpipe,non functional
4,19728,0.0,2011-07-13,other,0,Artisan,31.130847,-1.825359,Lake Victoria,Kagera,...,True,unknown,gravity,other,never pay,soft,seasonal,surface,communal standpipe,functional


# Differ from Su's, I save the 'longitude', 'latitude' 

# After the record time and population , we can tocsv and get a csv file.

In [78]:
# to csv

In [ ]:
# test for the 